The goal of this example is to show how to make a submission. This is a toy model and should probably not be taken as a starting point. It does not take into accound all the information or the features temporality.

> **Fork/edit this version to run the notebook and try the submission.**

## Import modules and loading the data
First let's import modules, the training and the test set. The test set contains the same columns than the training set without the target.

In [1]:
#import modules
from sklearn import metrics, model_selection
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
#loading the data
train = pd.read_csv('/kaggle/input/football-match-probability-prediction/train.csv')
test = pd.read_csv('/kaggle/input/football-match-probability-prediction/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:

train.head()

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

# Exploratory Data Analysis 

1. Number of Matches played every year 
2. Home Vs Away team wins ratio across years, across Leagues

In [4]:
data_eda=train

win_dict={'home':1,'away':-1,'draw':0}
data_eda['result_mapping']=data_eda['target'].map(win_dict)


# Combine Train and Test for Feature Engineering 

In [5]:
#keep only rating features
rating_features = [x for x in  train if 'rating' in x]

#Normalizing Features


# rating_features.append(feature)
    
    
y=data_eda['result_mapping']
combined_train=data_eda.drop(['result_mapping'],axis=1)
combined_train['tag']='Train_Data'

combined_test=test
combined_test['tag']='Test_Data'

combined=combined_train.append(combined_test)
combined['year']=pd.to_datetime(combined['match_date']).dt.year

import datetime as dt
combined['date_difference_last_match_home']=(pd.to_datetime(combined['match_date'])-pd.to_datetime(combined['home_team_history_match_date_1'])).dt.days
combined['date_difference_last_match_away']=(pd.to_datetime(combined['match_date'])-pd.to_datetime(combined['away_team_history_match_date_1'])).dt.days

combined['Goals_For_Home']=combined['home_team_history_goal_1']+combined['home_team_history_goal_2']+combined['home_team_history_goal_3']+combined['home_team_history_goal_4']+combined['home_team_history_goal_5']
combined['Opponent_Goals_For_Home']=combined['home_team_history_opponent_goal_1']+combined['home_team_history_opponent_goal_2']+combined['home_team_history_opponent_goal_3']+combined['home_team_history_opponent_goal_4']+combined['home_team_history_opponent_goal_5']
combined['goal_difference']=combined['Goals_For_Home']-combined['Opponent_Goals_For_Home']

combined['Goals_For_Away']=combined['away_team_history_goal_1']+combined['away_team_history_goal_2']+combined['away_team_history_goal_3']+combined['away_team_history_goal_4']+combined['away_team_history_goal_5']
combined['Opponent_Goals_For_Away']=combined['away_team_history_opponent_goal_1']+combined['away_team_history_opponent_goal_2']+combined['away_team_history_opponent_goal_3']+combined['away_team_history_opponent_goal_4']+combined['away_team_history_opponent_goal_5']
combined['goal_difference_away']=combined['Goals_For_Away']-combined['Opponent_Goals_For_Away']

#Performance in Last 5 matches
for i in range(1,6):
    combined['last_match_result_home_'+str(i)]=np.where(combined['home_team_history_goal_'+str(i)]>combined['home_team_history_opponent_goal_'+str(i)],3,np.where(combined['home_team_history_goal_'+str(i)]<combined['home_team_history_opponent_goal_'+str(i)],0,1))
    rating_features.append('last_match_result_home_'+str(i))
    
    combined['last_match_result_away_'+str(i)]=np.where(combined['away_team_history_goal_'+str(i)]>combined['away_team_history_opponent_goal_'+str(i)],0,np.where(combined['away_team_history_goal_'+str(i)]<combined['away_team_history_opponent_goal_'+str(i)],3,1))
    rating_features.append('last_match_result_away_'+str(i))

combined['home_performance_last_5']=combined['last_match_result_home_1']+combined['last_match_result_home_2']+combined['last_match_result_home_3']+combined['last_match_result_home_4']+combined['last_match_result_home_5']
combined['away_performance_last_5']=combined['last_match_result_away_1']+combined['last_match_result_away_2']+combined['last_match_result_away_3']+combined['last_match_result_away_4']+combined['last_match_result_away_5']
combined['home_vs_away']=combined['home_performance_last_5']+combined['away_performance_last_5']


rating_features=rating_features+['home_performance_last_5','away_performance_last_5','home_vs_away']

def streak(m1,m2,m3,m4,m5,k):
    streak=0
    
    lst_results=[m1,m2,m3,m4,m5]
    for i in lst_results:
        if(i==k):
            streak=streak+1
        else:
            return streak
            break
    
    return streak


combined['home_same_coach']=combined.apply(lambda x:streak(x.home_team_history_coach_1,x.home_team_history_coach_2,x.home_team_history_coach_3,x.home_team_history_coach_4,x.home_team_history_coach_5,x.home_team_coach_id),axis=1)
combined['away_same_coach']=combined.apply(lambda x:streak(x.away_team_history_coach_1,x.away_team_history_coach_2,x.away_team_history_coach_3,x.away_team_history_coach_4,x.away_team_history_coach_5,x.away_team_coach_id),axis=1)



mapping_freq=40

frequencies=combined['home_team_name'].value_counts()
mapping=combined['home_team_name'].map(frequencies)
combined['home_team_name_mapping']=combined['home_team_name'].mask(mapping<mapping_freq,'Other')

frequencies=combined['away_team_name'].value_counts()
mapping=combined['away_team_name'].map(frequencies)
combined['away_team_name_mapping']=combined['away_team_name'].mask(mapping<mapping_freq,'Other')

combined['away_coach_change']=np.where(combined['away_team_coach_id']==combined['away_team_history_coach_1'],0,1)


combined['home_team_coach_id']=combined['home_team_coach_id'].astype('str')
frequencies=combined['home_team_name'].value_counts()
mapping=combined['home_team_name'].map(frequencies)
combined['home_team_name_mapping']=combined['home_team_name'].mask(mapping<mapping_freq,'Other')


# Coach Mapping
combined['home_team_coach_id']=combined['home_team_coach_id'].astype('str')
frequencies=combined['home_team_coach_id'].value_counts()
mapping=combined['home_team_coach_id'].map(frequencies)
combined['home_team_coach_mapping']=combined['home_team_coach_id'].mask(mapping<mapping_freq,'Other')

combined['away_team_coach_id']=combined['away_team_coach_id'].astype('str')
frequencies=combined['away_team_coach_id'].value_counts()
mapping=combined['away_team_coach_id'].map(frequencies)
combined['away_team_coach_mapping']=combined['away_team_coach_id'].mask(mapping<mapping_freq,'Other')




combined['home_winning_streak']=combined.apply(lambda x:streak(x.last_match_result_home_1,x.last_match_result_home_2,x.last_match_result_home_3,x.last_match_result_home_4,x.last_match_result_home_5,3),axis=1)
combined['home_losing_streak']=combined.apply(lambda x:streak(x.last_match_result_home_1,x.last_match_result_home_2,x.last_match_result_home_3,x.last_match_result_home_4,x.last_match_result_home_5,0),axis=1)
combined['home_draw_streak']=combined.apply(lambda x:streak(x.last_match_result_home_1,x.last_match_result_home_2,x.last_match_result_home_3,x.last_match_result_home_4,x.last_match_result_home_5,1),axis=1)

combined['away_winning_streak']=combined.apply(lambda x:streak(x.last_match_result_away_1,x.last_match_result_away_2,x.last_match_result_away_3,x.last_match_result_away_4,x.last_match_result_away_5,3),axis=1)
combined['away_losing_streak']=combined.apply(lambda x:streak(x.last_match_result_away_1,x.last_match_result_away_2,x.last_match_result_away_3,x.last_match_result_away_4,x.last_match_result_away_5,0),axis=1)
combined['away_draw_streak']=combined.apply(lambda x:streak(x.last_match_result_away_1,x.last_match_result_away_2,x.last_match_result_away_3,x.last_match_result_away_4,x.last_match_result_away_5,1),axis=1)


combined

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

In [6]:
# import numpy as np
# combined['last_match_league']=combined.groupby(['league_name'])['match_date'].transform('max')
# combined['is_final']=np.where(combined['last_match_league']==combined['match_date'],1,0)
# combined.groupby(['home_team_coach_id'])['target'].value_counts()
# combined['home_team_coach_id'].value_counts()
print('Baseline is:')
print(combined['target'].value_counts())
combined.groupby(['home_same_coach'])['target'].value_counts()[0:40]



Baseline is:
home    48113
away    35174
draw    27651
Name: target, dtype: int64


home_same_coach  target
0                home      10868
                 away       8961
                 draw       6176
1                home       1574
                 away       1345
                 draw        988
2                home       1569
                 away       1260
                 draw        953
3                home       1474
                 away       1143
                 draw        902
4                home       1515
                 away       1100
                 draw        896
5                home      31113
                 away      21365
                 draw      17736
Name: target, dtype: int64

In [7]:
combined.columns

Index(['id', 'target', 'home_team_name', 'away_team_name', 'match_date',
       'league_name', 'league_id', 'is_cup', 'home_team_coach_id',
       'away_team_coach_id',
       ...
       'away_team_name_mapping', 'away_coach_change',
       'home_team_coach_mapping', 'away_team_coach_mapping',
       'home_winning_streak', 'home_losing_streak', 'home_draw_streak',
       'away_winning_streak', 'away_losing_streak', 'away_draw_streak'],
      dtype='object', length=226)

In [8]:
rating_features=rating_features+['goal_difference','goal_difference_away','Goals_For_Home','Goals_For_Away','is_cup','date_difference_last_match_home','home_team_name_mapping','away_team_name_mapping','home_team_coach_mapping','away_team_coach_mapping','away_coach_change','tag', 'home_winning_streak', 'home_losing_streak', 'home_draw_streak',
       'away_winning_streak', 'away_losing_streak','away_draw_streak','home_same_coach','away_same_coach']

# data_eda['league_score']=data_eda.groupby(['league_name'])['result_mapping'].transform('mean')
# data_eda['ht_score']=data_eda.groupby(['home_team_name'])['result_mapping'].transform('mean')
# data_eda['at_score']=data_eda.groupby(['away_team_name'])['result_mapping'].transform('mean')

In [9]:
# Missing Value Imputation 
from pandas.api.types import is_string_dtype
cols=combined.columns

for i in cols:
    if(is_string_dtype(combined[i])):
        combined[i]=combined[i].fillna(combined[i].mode()[0])
    else:
        combined[i]=combined[i].fillna(combined[i].median())

In [10]:
# Exploration 

combined[combined['tag']=='Train_Data']

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

In [11]:
# Train Data has More Home Team wins i.e 120k HT, 35k Away and 27k Draw
# Is Cup has less Draws possibly because of Knock-out matches
# Last 5 Match win performance is Important for both Home and Away teams

In [12]:
# combined.groupby(['is_cup'])['target'].value_counts()

# combined[combined['tag']=='Train_Data'][rating_features]['is_cup'].value_counts()

## Make a model
The model is very simple and use only rating data where missing values are filled with their mean. The a logistic regression is fitted. Since we predict the probabilites of home, away and draw is it a multiclass logistic regression.

In [13]:

# rating_features2=[x for x in  train if 'history' in x and 'date' not in x and 'rating'  not in x and 'league_id'  not in x and 'coach'  not in x ]
# rating_features=rating_features+rating_features2


combined1=combined[rating_features] 
combined1=pd.get_dummies(combined1)
#Make X and y

X_Train = combined1[combined1['tag_Train_Data']==1].drop(['tag_Train_Data','tag_Test_Data'],axis=1)
# X_Train=pd.get_dummies(X_Train)
y_Train = y

# # Sample Validation Data
# # n_test=1000
# # X_Train1=X_Train[n_test:]
# # y_Train1 = y[n_test:]

# # X_Train2=X_Train[0:n_test]
# # y_Train2 = y[0:n_test]

# #Test Data 
X_test = combined1[combined1['tag_Train_Data']==0].drop(['tag_Train_Data','tag_Test_Data'],axis=1)
# X_test=pd.get_dummies(X_test)


In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import lightgbm as lgb


# models=[KNeighborsClassifier(),LogisticRegression(),,DecisionTreeClassifier(max_depth=4),RandomForestClassifier(),GradientBoostingClassifier(),XGBClassifier()]
# labels=['KNN','Logistic','Decision_Tree','Random_Forest','Gradient_Boost','XGBoost']

models=[lgb.LGBMClassifier(num_leaves=42,learning_rate=0.01,n_estimators=400,objective='multiclass')]
labels=['XGBoost']

performance_final=pd.DataFrame()
for i in range(0,len(models)):
    model=models[i]
#     cv_results = cross_validate(model, data_x, y, cv  = StratifiedKFold(5, shuffle=True),return_train_score=True,scoring=model_metrics)
    
    cv_results = cross_validate(model, X_Train, y_Train, cv  = 2,return_train_score=True,scoring='neg_log_loss')
    
    metrics=pd.DataFrame(cv_results).mean().reset_index()
    metrics['model_name']=labels[i]
    performance=pd.DataFrame(metrics.pivot_table(index='model_name',columns='index', values=0).to_records())
    
    performance_final=performance_final.append(performance)
 

In [15]:
performance_final

,model_name,fit_time,score_time,test_score,train_score
0,XGBoost,46.788013,17.673844,-1.015281,-0.948579


In [16]:
model.fit(X_Train, y_Train)

LGBMClassifier(learning_rate=0.01, n_estimators=400, num_leaves=42,
               objective='multiclass')

In [17]:
# predictions_valid = model.predict(X_Train2[rating_features])
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_Train2, predictions_valid)
# cm

# Feature Importance 

In [18]:
# Feature Importance
# import difflib
# from sklearn.ensemble import RandomForestClassifier
# model=RandomForestClassifier()
# model.fit(X_Train, y_Train)
# fi = pd.DataFrame({'feature': list(X_Train.columns),
#                    'importance': 100*model.feature_importances_}).\
#                     sort_values('importance', ascending = False)
# fi.head(40)

In [19]:
#evaluate the model log loss using cross validation
# cv_scores = model_selection.cross_val_score(pipeline, X, y, cv=10,scoring='neg_log_loss')
# print(f'The cross validation log loss is {-cv_scores.mean().round(4)}')

In [20]:
# league_data=data_eda[['league_name','league_score']].drop_duplicates()
# # ht_data=data_eda[['home_team_name','ht_score']].drop_duplicates()
# # at_data=data_eda[['away_team_name','at_score']].drop_duplicates()

# test2=test.merge(league_data,how='left')

## Fit the model and a make submission
Once the model is fitted we make the **probabilities prediction**. Then we make the submission dataframe with **4 columns**. The columns home, away and draw contain probability while the column id contains the match id.

In [21]:
# pipeline.fit(X, y)

# X_Train, X_test = X_Train.align(X_test, join = 'inner', axis = 1)

predictions = model.predict_proba(X_test)

In [22]:
predictions.shape

(72711, 3)

In [23]:
submission = pd.DataFrame(predictions ,columns=['away','draw','home'])

#do not forget the id column
submission['id'] = test['id']

#submit!
submission.to_csv('submission2.csv', index=False)
submission.head()

,away,draw,home,id
0,0.278402,0.327887,0.393712,17761448
1,0.384973,0.289325,0.325702,17695487
2,0.328710,0.308525,0.362765,17715496
3,0.544732,0.266287,0.188981,17715493
4,0.219073,0.280756,0.500171,17715492


Now the submission file is written down, just click on the submit button on the top right box.